In [ ]:
import time
start_time = time.time()

!pip install faster-whisper

end_time = time.time()
print("Setup seconds used:", end_time-start_time)

In [ ]:
import logging

logging.basicConfig()
logging.getLogger("faster_whisper").setLevel(logging.DEBUG)

from faster_whisper import WhisperModel

model_size = "large-v3"

# Run on GPU with FP16
model = WhisperModel(model_size, device="cuda", compute_type="float16")

# or run on GPU with INT8
# model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
# or run on CPU with INT8
# model = WhisperModel(model_size, device="cpu", compute_type="int8")


# batched_model = BatchedInferencePipeline(model=model)

In [ ]:
LST_FOLDERS = ["/kaggle/input/ad01ogg"]
OUTPUT_FOLDER = r'/kaggle/working/output'
import os

for folder in LST_FOLDERS:
    for root, dirs, files in os.walk(folder):
        for name in files:
            path = os.path.join(root, name)
            rel_path = os.path.relpath(path, )

for folder_path in LST_FOLDERS:
    # 获取输入文件夹的名称
    input_folder_name = os.path.basename(folder_path)

    # 遍历文件夹
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # 获取文件的完整路径
            path = os.path.join(root, file)

            # 计算相对路径
            relative_path = os.path.relpath(path, folder_path)

            # 拼接输出路径
            output_relative_path = os.path.join(input_folder_name, relative_path)
            output_path = os.path.join(OUTPUT_FOLDER, input_folder_name, relative_path.replace(file, f"{os.path.splitext(file)[0]}.lrc"))

            print("Start file: ", file)
            start_time = time.time()
            last_text = ""
            segments, info = model.transcribe(path, beam_size=5)
            # print("Detected language '%s' with probability %f" % (info.language, info.language_probability))
            # segments, info = batched_model.transcribe(path, batch_size=16)
            lst_content = []
            for segment in segments:
                if segment.text == last_text:
                    continue
                
                # 将时间从秒转换为LRC格式所需的时间字符串
                start_minutes, start_seconds = divmod(segment.start, 60)
                end_minutes, end_seconds = divmod(segment.end, 60)

                # 格式化时间字符串
                start_time_str = f"{int(start_minutes):02d}:{int(start_seconds):02d}.{int((start_seconds % 1) * 100):02d}"
                end_time_str = f"{int(end_minutes):02d}:{int(end_seconds):02d}.{int((end_seconds % 1) * 100):02d}"

                lrc_line = f"[{start_time_str}] {segment.text}"
                last_text = segment.text
                lst_content.append(lrc_line)
                print(lrc_line)
                #print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

            end_time = time.time()
            print(path, "seconds used:", end_time-start_time)
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            with open(output_path, "w", encoding='utf-8') as f:
                f.write("\r\n".join(lst_content))

In [ ]:
import shutil
shutil.make_archive('/kaggle/working/result', 'zip', '/kaggle/working/output')

print("Finished")